## 스크랩한 내용 메일링 서비스 프로세스

1. 데이터 읽어오기
    - 오늘 스크랩한 내용 일자,항공사,노선별 최소,평균,최대값 읽어오기
2. HTML 작성
    - 읽어온 데이터와 템플릿을 이용 HTML 파일 작성
3. 메일 보내기
    - 메일링 대상자에게 메일 보내기

In [1]:
import sqlite3
import json
import datetime
from common.log_util import *
from common.mail_service_util import email_sender
## 로그 초기화
logger_initialize('mail_logger_setting.json')

In [2]:
## 데이터 읽어오기
def read_fares(db,sitecodes,dpt,arr,scrap_date=None):
    if scrap_date is None: #가장 최근 정보 읽어오기
        conn = sqlite3.connect(db)
        with conn:
            cur = conn.cursor()
            sql = "select max(scrap_date) from airfare_scraped_data where scrap_site='{}' and dpt='{}' and arr='{}'"
            cur.execute(sql.format(sitecode,dpt,arr))
            scrap_date = cur.fetchone()[0]
        
    fares = []
    conn = sqlite3.connect(db)
    with conn:
        cur = conn.cursor()
        sitecode = ' or '.join(["scrap_site='"+c+"'" for c in sitecodes])
        sql = '''select dpt_date,airline,min(fare),avg(fare),max(fare) from airfare_scraped_data
                where scrap_date='{scrap_date}' and ({sitecode}) and dpt='{dpt}' and arr='{arr}' and fare != '' and fare > 0
                group by airline,dpt_date
                order by airline,dpt_date'''
        #print(sql.format(scrap_date=scrap_date,sitecode=sitecode))
        cur.execute(sql.format(scrap_date=scrap_date,sitecode=sitecode,dpt=dpt,arr=arr))
        rows = cur.fetchall()
        for row in rows:
            fares.append([row[0], row[1], row[2], row[3], row[4]])
    return fares,scrap_date
## 템플릿 이용 HTML 파일 작성
def make_html(fare_data,sitecodes,dpt,arr,scrap_date):
    ## 검색조건 만들기
    cond_data = "대상사이트 : {}<br>노선 : {} 60일<br>{} 스크랩핑 데이터".format('/'.join(sitecodes),dpt+'-'+arr,scrap_date)
    with open('template/fare_linechart_monoton.html',encoding='utf-8') as fp:
        template = fp.read()
        chart_data = make_chart_data(fare_data)
        grid_data = make_grid_data(fare_data)
    return template.replace('{condition_data}',cond_data).replace('{chart_data}',chart_data).replace('{grid_data}',grid_data).replace('\n','')

## chart 데이터 작성
def make_chart_data(fare_data):
    chart_data = {}
    for fare in fare_data:
        fare_id = fare[1]+'Min' 
        fare_values = {'date':fare[0],'fares':int(fare[2])}
        chart_data[fare_id]= chart_data.get(fare_id,[])
        chart_data[fare_id].append(fare_values)
        fare_id = fare[1]+'Avg' 
        fare_values = {'date':fare[0],'fares':int(fare[3])}
        chart_data[fare_id]= chart_data.get(fare_id,[])
        chart_data[fare_id].append(fare_values)
        fare_id = fare[1]+'Max' 
        fare_values = {'date':fare[0],'fares':int(fare[4])}
        chart_data[fare_id]= chart_data.get(fare_id,[])
        chart_data[fare_id].append(fare_values)
    chart_list = []
    for data_id,data_values in chart_data.items():
        chart_list.append(dict((('id',data_id),('values',data_values))))
    #return str(chart_list).replace("'",'"')
    return json.dumps(chart_list)
## Grid 데이터 작성
def make_grid_data(fare_data):
    grid_html = ''
    tr_html = '<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>'
    for fare in fare_data:
        grid_html += tr_html.format(fare[0],fare[1],int(fare[2]),int(fare[3]),int(fare[4]))
    return grid_html

## 발송처리
def mail_service(db,receivers,service_code,mail_msg):
    log('start email service')
    ## 발송대상 파일 생성
    files = []
    for code in service_code:
        sitecodes = code['sitecodes']
        dpt = code['dpt']
        arr = code['arr']
        fare_data,scrap_date=read_fares('airfare_scraped_data.db',sitecodes,dpt,arr)
        html = make_html(fare_data,sitecodes,dpt,arr,scrap_date)
        html_file = 'html/report_{}.html'.format(datetime.datetime.today().strftime('%Y%m%d%H%M%S%f'))
        # print(html_file)
        with open(html_file,'wt',encoding='utf-8') as fp:
            fp.write(html)
        files.append(html_file)
    log('make attach files')
    log(files)
    # html 메시지 생성
    with open('template/newletter.html',encoding='utf-8') as fp:
        template = fp.read()
    ## 수정 및 추가 사항 처리
    template= template.replace('{msg_title1}',mail_msg['title1']).replace('{msg_title2}',mail_msg['title2'])
    template = template.replace('{msg_date}',mail_msg['date']).replace('{msg_auth}',mail_msg['auth'])
    template = template.replace('{msg_contents}',mail_msg['contents']).replace('\n','')
    #html = template
    log('start email sender')
    for email in receivers:
        log(['receiver',email])
        email_sender(email,'rms reporting service',template,files)
    log('end email service')

### 메일 대상 관리 수정
- 파일 작성시 작성된 파일과 노선을 매핑
    * 파일명도 의미를 부여하여 일/기간별 파일과 일/플라이트별 파일이 구분가도록 처리
    * 파일명 처리 고민 필요?
    * 파일 생성과 메일링 분리 검토
- 각 대상별 메일대상 파일 가능하도록 수정
    * 수신자와 매핑할 정보 처리 고민?

In [4]:
## 1차 서비스 범위 - IP, ZE/TW/7C, GMP-CJU, 60일
service_code = [
    { "sitecodes":['IP'],"dpt":"GMP","arr":"CJU" },
    { "sitecodes":['IP'],"dpt":"ICN","arr":"NRT" },
    { "sitecodes":['7C','LJ','TW','BX','ZE'], "dpt":"GMP", "arr":"CJU" }
]
mail_msg = {
    'title1' : 'RMS 뉴스 레터 - 온라인 항공사 가격 정보[발송처리 변경]',
    'title2' : 'GMP-CJU(인터파크,항공사) / ICN-NRT(인터파크) 60일 온라인 가격 현황',
    'date' : datetime.datetime.today().strftime("%Y-%m-%d"),
    'auth' : 'park363@eastarjet.com',
    'contents' : '''
    <p>온라인 상의 항공사 가격 동향 입니다.</p>
    <p>작성일을 기준으로 다음날 부터 60일 동안의 가격을 읽어와 1일 기준 최소값 평균값 최대값을 계산하였습니다.</p>
    <p>전체 자료가 필요하신 분은 요청하시면 CSV 형태로 제공해 드리겠습니다.</p>
    <p>해당 노선 서비스 테스트후 대상 노선 및 항공사 추가 진행 하도록 하겠습니다.</p>
    <p>메일링 추가가 필요하신 분은 저에게 메일을 주시기 바랍니다.</p>
    <br><br><p style="font-weight:bold;">[주의사항] html 파일을 첨부하였습니다. </p>
    <p>아웃룩 사용자는 첨부 파일이 보이지 않을 수 있습니다. 그룹웨어에서 확인하시기 바랍니다.</p>
    <p>익스플로어 하위 버전에서 표출이 안될 수 도 있습니다. 가능하면 크롬 브라우저를 사용하시기 바랍니다.</p>
    <br><br><p style="font-weight:bold;">[발송처리 변경] 추가 개발중</p>
    <p>발송 대상자별로 목적에 맞는 파일만 발송 할 수 있도록 변경 처리 개발중</p>
    <p>각 업무에 맞는 파일만 보내달라는 요청이 있어 발송시 체크 할 수 있도록 변경진행중 입니다.(이석규팀장 요청)</p>
    <p>추가 요청 사항에 대한 의견이 있으면 김희정과장님 통해 요청해 주시기 바랍니다.</p>
    <p>가능한 반영하도록 하겠습니다.</p>
    <p>오전 세미나로 조금 늦었습니다.</p>
    '''
}
target = [
    '박종욱<park363@eastarjet.com>','우성주<topatsee@eastarjet.com>','조용현<ycho1211@eastarjet.com>','염희석<hsyoum@eastarjet.com>',
    '김희정<amykim@eastarjet.com>','정희진<tokkaipi@eastarjet.com>','안병준<bjahn@eastarjet.com>','장혜란<jhyeran@eastarjet.com>',
    '이동현<ldh1@eastarjet.com>','이재은<leeje@eastarjet.com>','임서윤<lhj01@eastarjet.com>','황혜진<hhj@eastarjet.com>',
    '이석규<sklee@eastarjet.com>','안경미<km9980@eastarjet.com>','권재혁<jhkwon@eastarjet.com>'
]
test = ['박종욱<park363@eastarjet.com>']
receivers = test
mail_service('airfare_scraped_data.db',receivers,service_code,mail_msg)

2017-06-27 10:57:14,154 root     DEBUG    start email service
2017-06-27 10:57:15,332 root     DEBUG    make attach files
2017-06-27 10:57:15,335 root     DEBUG    html/report_20170627105714544101.html :: html/report_20170627105714916122.html :: html/report_20170627105715314145.html
2017-06-27 10:57:15,338 root     DEBUG    start email sender
2017-06-27 10:57:15,341 root     DEBUG    receiver :: 박종욱<park363@eastarjet.com>
2017-06-27 10:57:15,344 root     DEBUG    receiver :: 박종욱<park363@eastarjet.com> ::  ## email - :: rms reporting service
2017-06-27 10:57:16,122 root     DEBUG    receiver :: 우성주<topatsee@eastarjet.com>
2017-06-27 10:57:16,125 root     DEBUG    receiver :: 우성주<topatsee@eastarjet.com> ::  ## email - :: rms reporting service
2017-06-27 10:57:16,822 root     DEBUG    receiver :: 조용현<ycho1211@eastarjet.com>
2017-06-27 10:57:16,830 root     DEBUG    receiver :: 조용현<ycho1211@eastarjet.com> ::  ## email - :: rms reporting service
2017-06-27 10:57:17,580 root     DEBUG    rec